In [1]:
from os import sys
sys.path.append("../../meshAfterParty/")
import trimesh
import skeleton_utils as sk
import neuron
from importlib import reload

In [2]:
segment_id = 92700513338137592
current_mesh_file = "/notebooks/neuron_mesh_tools/test_neurons/spine_detection/92700513338137592_excitatory_3.off"
description="inhibitory_1"

In [3]:
current_neuron = trimesh.load_mesh(current_mesh_file)
sk.graph_skeleton_and_mesh(other_meshes = [current_neuron])

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [4]:
my_neuron = neuron.Neuron(
    mesh=current_neuron,
    segment_id=segment_id,
    description=description
                         )

--- 0) Having to preprocess the Neuron becuase no preprocessed data
Please wait this could take a while.....


--- 0) Total time for preprocessing: 426.7117795944214




--- 1) Finished unpacking preprocessed materials: 6.67572021484375e-06
total_edges = [['S0', 'L0'], ['S0', 'L1'], ['S0', 'L2'], ['S0', 'L3'], ['S0', 'L4'], ['S0', 'L5'], ['S0', 'L6'], ['S0', 'L7'], ['S0', 'L8'], ['S0', 'L9'], ['S0', 'L10'], ['S0', 'L11']]
--- 2) Finished creating neuron connectivity graph: 0.00013947486877441406
Having to generate soma_meshes_face_idx because none in preprocessed data
--- 3a) Finshed generating soma_meshes_face_idx: 0.25740981101989746
Using Poisson Surface Reconstruction to make mesh watertight
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 5702
xvfb-run -n 5702 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/neuron_mesh_tools/meshAfterParty/development/Poisson_temp/neuron_82493.off -o /notebooks/neuron_mesh_tools/meshAfterParty/development/Poisson_temp/neuron_82493_poisson.off -s /notebooks/neuron_mesh_tools/meshAfterParty/development/Poisson_temp/poisson_89

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!


removed temporary input file: /notebooks/neuron_mesh_tools/meshAfterParty/development/Poisson_temp/neuron_82493.off
removed temporary output file: /notebooks/neuron_mesh_tools/meshAfterParty/development/Poisson_temp/neuron_82493_poisson.off
/notebooks/neuron_mesh_tools/meshAfterParty/development/Poisson_temp/poisson_891770.mls is being deleted....
--- 3) Finshed generating soma objects and adding them to concept graph: 10.558424234390259
--- 4a) Finshed generating curr_limb_meshes_face_idx: 1.0582811832427979
checking and resolving cycles
No cycles to fix
checking and resolving cycles
No cycles to fix
checking and resolving cycles
No cycles to fix
checking and resolving cycles
No cycles to fix
checking and resolving cycles
No cycles to fix
checking and resolving cycles
No cycles to fix
checking and resolving cycles
No cycles to fix
Concept graph size was 1 or less so returning original
Concept graph size was 1 or less so returning original
Concept graph size was 1 or less so returning 

# Practice saving off neuron

In [7]:
import system_utils as su
su = reload(su)
my_neuron.save_neuron_object()

No filename/location given so creating own
Saving Object at: /notebooks/neuron_mesh_tools/meshAfterParty/development/92700513338137592_inhibitory_1.pkl


In [8]:
loaded_neuron = su.load_object("/notebooks/neuron_mesh_tools/meshAfterParty/development/92700513338137592_inhibitory_1.pkl")

# Updating the functions of Neuron

In [10]:
neuron = reload(neuron)
updated_neuron = neuron.Neuron(loaded_neuron,minimal_output=True)

Processing Neuorn in minimal output mode...please wait
Total time for neuron instance creation = 14.066520690917969


# Plotting the parts that are merge errors

In [77]:
"""
For every limb
1) Make a copy of the concept_network
2) Get all of the branches that are below a certain threshold
3) Delete these branches from the network
4) Get all of the branch pieces that are not a part of the component with the starting coordinate
5) collect a list of those branch names (will use to visualize later)

To visualize:
For each limb:
1) collect the meshes of all of the branches in the merge list
2) Add to total list to view

"""
import networkx as nx
import matplotlib.pyplot as plt
import networkx_utils as xu
import numpy as np
import numpy_utils as nu
import neuron_visualizations as nviz
from tqdm.notebook import tqdm

nviz = reload(nviz)
nu = reload(nu)
current_neuron = updated_neuron
merge_width_threshold = 400


limb_list = current_neuron.get_limb_node_names()

limb_merge_branches_list = dict()
total_merge_meshes = []

for curr_limb in tqdm(limb_list):

    #1) Get all of the branches that are below a certain threshold
    curr_concept_network = current_neuron.concept_network.nodes[curr_limb]["data"].concept_network.copy()

    #nx.draw(curr_concept_network,with_labels=True)

    #2) Get all of the branches that are below a certain threshold
    below_threshold_nodes = [k for k in curr_concept_network.nodes() if curr_concept_network.nodes[k]["data"].width < merge_width_threshold]

    #visualize the branch and the parts to remove
    curr_limb_mesh = current_neuron.concept_network.nodes[curr_limb]["data"].mesh
    below_threshold_meshes = [curr_concept_network.nodes[k]["data"].mesh for k in below_threshold_nodes]
    # sk.graph_skeleton_and_mesh(main_mesh_verts=curr_limb_mesh.vertices,
    #                           main_mesh_faces=curr_limb_mesh.faces,
    #                           other_meshes=below_threshold_meshes,
    #                           other_meshes_colors="red")

    #3) Delete these branches from the network
    curr_concept_network.remove_nodes_from(below_threshold_nodes)
    seperated_components = list(nx.connected_components(curr_concept_network))

    #3b) Make sure that the original concept graph had a starting component
    try:
        xu.get_starting_node(curr_concept_network,attribute_for_start="starting_coordinate")
    except:
        raise Exception(f"Original {curr_limb} starting graph did not have a starting_coordinate")

    subcomponent_with_starting_node = -1
    non_starter_components = []
    for j,sp_comp in enumerate(seperated_components):
        curr_subgraph = curr_concept_network.subgraph(sp_comp)
        try: 
            starting_component = xu.get_starting_node(curr_subgraph)
            subcomponent_with_starting_node = j
        except:
            non_starter_components.append(list(sp_comp))

    #print(f"subcomponent_with_starting_node={subcomponent_with_starting_node}")

    if subcomponent_with_starting_node == -1:
        raise Exception("Starting Node was below threshold")
    
    total_merge_pieces_leftover = nu.concatenate_lists(non_starter_components)
        
    total_merge_pieces = nu.concatenate_lists([below_threshold_nodes,total_merge_pieces_leftover])

    merge_meshes = [current_neuron.concept_network.nodes[curr_limb]["data"].concept_network.nodes[k]["data"].mesh for k in total_merge_pieces]

#     sk.graph_skeleton_and_mesh(main_mesh_verts=curr_limb_mesh.vertices,
#                               main_mesh_faces=curr_limb_mesh.faces,
#                               other_meshes=merge_meshes,
#                               other_meshes_colors="red")
    
    limb_merge_branches_list[curr_limb] = total_merge_pieces
    

nviz.plot_branch_pieces(current_neuron.concept_network,
                       node_to_branch_dict=limb_merge_branches_list,
                      background_mesh=current_neuron.mesh,
                       mesh_alpha = 0.8,
                       html_path=f"{current_neuron.segment_id}_{current_neuron.description}_merge_segments.html")

In [59]:
total_merge_pieces

# How to create a neuron with former preprocessed data

In [ ]:
# import time
# neuron_object_time = time.time()

# Neuron_obj = neuron.Neuron(mesh=current_neuron,preprocessed_data=preprocessed_data)

# print(f"Total time for Neuron object creation = {time.time() - neuron_object_time}")

# how to initialize neuron with other neuron

In [ ]:
new_neuron = neuron.Neuron(my_neuron,minimal_output=True)

# Plotting the concept network of limb of a neuron

In [ ]:
import neuron_visualizations as nviz
nviz=reload(nviz)
new_neuron.plot_limb_concept_network("L1",node_size=5,starting_node_size=8,arrow_size=5)

# Plotting the concept network of whole neuron limbs/soma

In [ ]:
neuron = reload(neuron)
new_neuron.plot_soma_limb_concept_network(node_colors={"L1":"brown"})

In [ ]:
su = reload(su)

from neuron import Neuron
another_neuron = Neuron(new_neuron)
su.save_object(another_neuron,"inhibitory_saved_neuron")

# Practicing the basic big visualization functions

In [ ]:
#plotting the mesh
import skeleton_utils as sk
sk = reload(sk)

curr_limb_idx = 2

"""
Pseudocode: 
1) Get the concept map from the concept_map["L" + limb_idx]["data"].concept_map
2) Iterate through all the nodes and get the total meshes and skeletons
3) Graph these at random
"""

random_colors_used = sk.graph_skeleton_and_mesh(
    other_meshes=[v["branch_mesh"] for k,v in limb_correspondence[curr_limb_idx].items() ],
                          other_meshes_colors="random",
                          other_skeletons=[v["branch_skeleton"] for k,v in limb_correspondence[curr_limb_idx].items() ],
                          other_skeletons_colors="random",
                            mesh_alpha=0.8,
                          return_other_colors=True)
random_colors_used_no_alpha = random_colors_used[:,:-1]

In [ ]:
type(my_neuron)

# Method for finding and visualizing merge errors

In [ ]:
segment_id = 12345
current_mesh_file = "/notebooks/neuron_mesh_tools/test_neurons/multi_soma_example.off"
description="2_somas_with_merge"

double_soma_neuron = trimesh.load_mesh(current_mesh_file)
sk.graph_skeleton_and_mesh(other_meshes = [double_soma_neuron])

In [ ]:
double_neuron = neuron.Neuron(
    mesh=double_soma_neuron,
    segment_id=segment_id,
    description=description
                         )

In [ ]:
"""
How to get merge errors

"""